In [2]:
from generate_data import parse_input

tmp = parse_input()

    

⠙ Parsing input files ...

In [13]:
import polars as pl 
import pandas as pd

df = pl.DataFrame({"foo": [1, 2, 3], "bar": [-1, 5, 8]})
df.to_pandas()

ValueError: Must pass schema, or at least one RecordBatch